## Lecture 6 (Exotics Markets) Assignment, MTH 9865

Due start of class, October 14, 2015

#### Student: Weiyi Chen

### Question 1 (6 marks)

In class we looked at Gaussian copulas for pricing two-asset derivatives. We talked about using that model to price cross-pair options based on the two USD-pair option markets, calibrating the Gaussian copula correlation parameter such that the model reproduces the ATM cross option price.

When we do that, the model tends to underprice the butterfly – ie the implied volatility smile that comes out of the model is lower than the market. We said that was due to the fact that the model does not include any premium for stochastic correlation, which it should because correlation is not constant.

A stochastic correlation only affects the value of a derivative if its exposure to that correlation is non-linear. Let’s consider a specific case of the Gaussian copula to examine that: one where the RR and BF of the USD pairs is zero. In that case, their pricing is just Black-Scholes pricing, and if we assume a constant correlation, the pricing of the cross option is Black-Scholes with an implied volatility

$$
\sigma_x = \sqrt{\sigma_1^2 + \sigma_2^2 - 2\rho\sigma_1\sigma_2}
$$

Calculate the “gamma” of the cross option price with respect to the correlation parameter . Assume a market where the two USD-pair spots are 1, interest rates are zero, time to expiration is 0.5y, the USD-pair volatilities are both equal to 10%, and the correlation is +25%, and plot the correlation gamma as a function of strike for the cross-pair options.

Discuss the qualitative impact stochastic correlation should have on the cross-pair implied volatilities based on that plot.



#### Solution:

The "delta" is 

$$
\begin{split}
\frac{\partial C}{\partial P} &= \frac{\partial C}{\partial \sigma_x} \frac{\partial \sigma_x}{\partial P} \\
&= S\sqrt{\frac{T}{2\pi}} \left[ e^{-\frac{d_1^2}{2}} (-\frac{\sigma_1\sigma_2}{\sigma_x})\right]
\end{split}
$$

The "gamma" is

$$
\begin{split}
\frac{\partial^2 C}{\partial P^2} = S\sqrt{\frac{T}{2\pi}}  \frac{\partial\left[ e^{-\frac{d_1^2}{2}} (-\frac{\sigma_1\sigma_2}{\sigma_x})\right]}{\partial P}
\end{split}
$$

We finally derive

$$ 
\frac{\partial^2 C}{\partial P^2} = S\sqrt{\frac{T}{2\pi}} \left[ e^{-\frac{d_1^2}{2}} (-\frac{\sigma_1^2\sigma_2^2}{\sigma_x^2})\right]\left[d_1 (\frac{1}{2}\sqrt{T}-\frac{\ln(S/K)}{\sigma^2\sqrt{T}}) + \frac{1}{\sigma_x} \right]
$$

By running the function `run_corr_gamma` in baruch/assig6.py, we derive - 

<img src='corr_gamma.png', width=600>

The correlation gamma is negative for ATM cross options and positive for OTM cross options. This means that any volatility tends to push down the prices of ATM cross options while push up the prices of OTM cross options, resulting in the increasing of the cross butterfly.

In [ ]:
# code can be run in wst environment

class CorrOption(gromit.Object):
    
    @gromit.CanSet
    def Spot1(self):
        return 1.
    
    @gromit.CanSet
    def Spot2(self):
        return 1.
    
    @gromit.CanSet
    def Vol1(self):
        return .1
    
    @gromit.CanSet
    def Vol2(self):
        return .2
    
    @gromit.CanSet
    def Rho(self):
        return .25
    
    @gromit.CanSet
    def StrikeX(self):
        return 0.
    
    @gromit.CanSet
    def Texp(self):
        return .5
    
    @gromit.CanSet
    def Rho_eps(self):
        return 1e-4
    
    @gromit
    def VolX(self, key):
        if key == 'mid':
            return math.sqrt(self.Vol1()**2+self.Vol2()**2-2*self.Rho()*self.Vol1()*self.Vol2())
        elif key == 'up':
            return math.sqrt(self.Vol1()**2+self.Vol2()**2-2*(self.Rho()+self.Rho_eps())*self.Vol1()*selv.Vol2())
        elif key == 'down':
            return math.sqrt(self.Vol1()**2+self.Vol2()**2-2*(self.Rho()-self.Rho_eps())*self.Vol1()*selv.Vol2())
        
    @gromit
    def OptX(self, key):
        spotx = self.Spot1() / self.Spot2()
        return bs.opt_price(strikex>=spotx,spotx,self.StrikeX,self.Texp,self.VolV(key),0,0)
    
    def Gamma(self):
        return (self.OptX('up') + self.OptX('down') - 2 * self.OptX('mid')) / self.Rho_eps**2

### Question 2 (3 marks)

Describe the market dynamic that is most important for knockout out pricing, and compare that to the market dynamic that is the most important for volatility swap pricing.

For each, explain why that market dynamic is important to the pricing.


#### Solution

The most important market dynamic for knockout out pricing is the risk reversal beta, and the most important for volatility swap pricing is the realized volatility of implied volatility.

##### Knockout out pricing

For knockouts, we can construct a hedge portfolio of vanilla options that is a “semi-static” hedge. If long a down-and-out knockout call, we can sell the regular call and buy a put whose strike is $\frac{B_2}{K}$. That portfolio hedges almost all my market risk unless the barrier is hit – then I need to unwind it. 

The cost of unwinding is mostly a function of risk reversal, because at that point the call and put are equally out-of-the-money. Their net sensitivity to ATM vol is small, which is sensitive only to the level of risk reversal. The expected level of risk reversal is how much risk reversal will have moved while spot moved to the barrier, which is our measure of risk reversal beta: the regression coefficient of moves in risk reversal with moves in spot.

##### Volatility swap pricing

For vol swaps, the hedge for a long vol swap is a short variance swap. The vol swap looks like a square-root payoff against the variance swap, which is non-linear dependence. Keep rebalancing the notional of the short variance swap.

You expect to lose money doing so, as it’s a short gamma position. You then expect to lose money over time running a long vol swap position, and you get compensated for that by being able to enter the vol swap at a fair strike that’s less than the square root of the variance swap fair strike.  The discount in fair strike depends on the realized volatility of the variance swap fair strike, which is roughly the same as realized volatility of implied volatility.


### Question 3 (4 marks)

Consider a dual digital option that pays \$1 if EURUSD is above a strike $K_1$ and GBPUSD is above a strike $K_2$. All discount rates are zero. The price of the EURUSD European digital option (paying \$1 if EURUSD is above $K_1$) is $65\%$ and the price of the GBPUSD European digital option (paying \$1 if GBPUSD is above $K_2$) is $30\%$.

Plot the price of the dual digital option priced under a Gaussian copula model, for correlation parameter ranging from $-100\%$ to $+100\%$. Qualitatively explain the behavior of the price sensitivity to correlation.

#### Solution

That the first step is translating both the two strikes into standard normal variables,

$$ N(x) = F(K) $$

where $N(x)$ is the standard cumulative normal distribution function and $F(K)$ is the probability that the spot is less than or equal to some level .

For the EURUSD piece, we know the price of a digital call $1-F(K)$, then

$$ x_1 = N^{-1}(1 - 0.65) = -0.3853 $$

Similarly for the GBPUSD,

$$ x_2 = N^{-1}(1 - 0.30) = 0.5244 $$

Then we can proceed to calculate the joint probability by using the bivariate standard normal cumulative distribution function. Plot the price of the dual digital option priced under a Gaussian copula model (by running the function `run_dual_digi` in baruch/assig6.py

<img src='dual_digi.png', width=600>

The price increases with correlation, which is as expected: if the first asset is above the strike, when the correlation is positive, the second asset is more like to be above the barrier as well.

In [ ]:
class DualDigital(gromit.Object):
    
    @gromit.CanSet
    def IsCall(self):
        True
        
    @gromit.CanSet
    def Price(self, key):
        if key == 1:
            return .65
        elif key == 2:
            return .3
    
    @gromit.CanSet
    def Rho(self):
        return 0.
    
    @gromit
    def F(self, key):
        return self.Price(key) if not self.IsCall() else 1-self.Price(key)
    
    @gromit
    def X(self, key):
        return num.cnorminv(F(key))
    
    @gromit
    def DualDigi(self):
        if self.IsCall():
            return scipy.stats.mvn.mvndst([self.X(1),self.X(2)],[0,0],[1,1],[self.Rho()])[1]
        else:
            return scipy.stats.mvn.mvndst([0,0],[self.X(1),self.X(2)],[0,0],[self.Rho()])[1]

### Question 4 (10 marks)

Investigate knockout pricing under the LV/SV approximation model.

#### Solution

By running the function `run_lvsrun_lvs` in baruch/assig6.py, we are able to generate plot

<img src='knockout_price.png', width=600>

for knockout price vs. $\alpha$. The typical bid/ask spread for knockouts is only a few basis points, so the impact of changing the model parameter is very significant.

And plot

<img src='vanilla_price.png', width=600>

for vanilla price vs. $\alpha$. As expected the vanilla option price hardly varies at all, because for each different value of $\alpha$ we’re recalibrating the model to hit vanilla prices again.

(Did not really write the codes for this part, as Mark provided it before the exam/due, and I've reviewed it to prepare